<a href="https://colab.research.google.com/github/ikanx101/G-Colab/blob/main/Kelas_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [55]:
library(dplyr)
library(ompr)
library(ompr.roi)
library(ROI.plugin.glpk)

In [56]:
rm(list=ls())
n = 167
m = 12 # ada 4 kelas
pilih = 3 # pilih 3 batch
capacity = rep(16,m) # rep.int(round(n/m,0)+1,m)

set.seed(101)
pilihan = list()
for(i in 1:n){
  pilihan[i] = list(sample(m,pilih))
}

pilihan

[[1]]
[1]  9 12  7

[[2]]
[1]  1 10  6

[[3]]
[1]  3 12  9

[[4]]
[1]  3 12  2

[[5]]
[1] 4 5 1

[[6]]
[1] 12  1  6

[[7]]
[1]  8 10  5

[[8]]
[1] 10 12  5

[[9]]
[1] 10  5  4

[[10]]
[1]  8 12  4

[[11]]
[1] 6 8 7

[[12]]
[1] 10 12  9

[[13]]
[1] 10  9  5

[[14]]
[1]  1  3 10

[[15]]
[1]  9  3 10

[[16]]
[1] 4 2 1

[[17]]
[1] 12  6  9

[[18]]
[1] 12  2 11

[[19]]
[1]  1  6 11

[[20]]
[1]  2  9 10

[[21]]
[1] 5 2 9

[[22]]
[1] 2 5 3

[[23]]
[1] 11  9 10

[[24]]
[1] 7 4 1

[[25]]
[1] 11  8  4

[[26]]
[1]  8 10 11

[[27]]
[1]  4 12  3

[[28]]
[1] 12 10  7

[[29]]
[1]  9 12 11

[[30]]
[1] 12  5  2

[[31]]
[1]  7  9 10

[[32]]
[1]  5  8 12

[[33]]
[1]  6  9 12

[[34]]
[1]  9  3 10

[[35]]
[1]  7 11  4

[[36]]
[1] 12  6  3

[[37]]
[1] 12  5  7

[[38]]
[1]  2  4 11

[[39]]
[1] 11  9 12

[[40]]
[1] 11  3  7

[[41]]
[1] 8 3 2

[[42]]
[1] 6 1 4

[[43]]
[1]  8  3 12

[[44]]
[1] 1 9 5

[[45]]
[1]  5  4 10

[[46]]
[1] 9 8 7

[[47]]
[1] 7 1 5

[[48]]
[1] 11 12  5

[[49]]
[1] 12 11  4

[[50]]
[1]  1  6 12

[[51]]
[1] 5 8 9

[[52]]
[1]  5 11 12

[[53]]
[1]  9  3 12

[[54]]
[1]  6 10 11

[[55]]
[1] 12  8  5

[[56]]
[1] 6 9 4

[[57]]
[1] 4 3 2

[[58]]
[1] 10  3  6

[[59]]
[1]  7 11  1

[[60]]
[1] 7 3 8

[[61]]
[1]  2 11  4

[[62]]
[1] 10  1  2

[[63]]
[1]  6  8 11

[[64]]
[1]  7 12  5

[[65]]
[1] 12  9  1

[[66]]
[1] 12  5  1

[[67]]
[1] 11  1  4

[[68]]
[1] 10  6  9

[[69]]
[1]  2  1 12

[[70]]
[1] 12  8 11

[[71]]
[1] 12  7  6

[[72]]
[1] 5 9 6

[[73]]
[1]  6  5 11

[[74]]
[1]  8 12 11

[[75]]
[1] 12  5  2

[[76]]
[1] 10  7  5

[[77]]
[1] 7 1 8

[[78]]
[1] 9 1 4

[[79]]
[1] 4 7 3

[[80]]
[1] 11  1  7

[[81]]
[1] 10  9  6

[[82]]
[1] 10  9  8

[[83]]
[1] 7 9 3

[[84]]
[1] 4 8 7

[[85]]
[1] 7 6 8

[[86]]
[1] 9 7 4

[[87]]
[1] 3 9 4

[[88]]
[1]  3  1 10

[[89]]
[1]  3  9 12

[[90]]
[1] 12  3  4

[[91]]
[1] 2 7 9

[[92]]
[1] 4 2 8

[[93]]
[1]  8 12  6

[[94]]
[1]  9 10  4

[[95]]
[1]  9  4 12

[[96]]
[1] 5 7 9

[[97]]
[1]  1  7 10

[[98]]
[1] 10  2  6

[[99]]
[1] 10 12  7

[[100]]
[1] 5 8 9

[[101]]
[1] 10  7 12

[[102]]
[1] 12  8  3

[[103]]
[1] 12 11  6

[[104]]
[1] 10 11 12

[[105]]
[1] 5 2 9

[[106]]
[1]  5  3 12

[[107]]
[1]  1  6 10

[[108]]
[1] 10  4 11

[[109]]
[1] 5 9 3

[[110]]
[1]  2 10  1

[[111]]
[1]  5 11 12

[[112]]
[1] 9 5 8

[[113]]
[1] 4 9 2

[[114]]
[1] 6 2 5

[[115]]
[1]  7  9 12

[[116]]
[1]  2 12  7

[[117]]
[1]  4 11  1

[[118]]
[1] 7 3 5

[[119]]
[1]  8 10  2

[[120]]
[1] 7 1 6

[[121]]
[1] 8 5 2

[[122]]
[1] 1 2 9

[[123]]
[1]  1 10  6

[[124]]
[1] 12 11  5

[[125]]
[1] 5 7 6

[[126]]
[1] 1 7 8

[[127]]
[1]  5 10  8

[[128]]
[1] 9 3 4

[[129]]
[1] 7 2 6

[[130]]
[1]  8 11 10

[[131]]
[1]  8 11 10

[[132]]
[1] 1 4 8

[[133]]
[1]  1  8 11

[[134]]
[1]  1 10 12

[[135]]
[1]  7 12  3

[[136]]
[1]  1 11 12

[[137]]
[1] 4 2 7

[[138]]
[1] 11  7 10

[[139]]
[1]  8  3 11

[[140]]
[1] 9 7 1

[[141]]
[1] 1 5 8

[[142]]
[1] 3 1 7

[[143]]
[1]  7 10  6

[[144]]
[1] 11 12  7

[[145]]
[1] 7 9 3

[[146]]
[1] 9 3 4

[[147]]
[1] 6 7 5

[[148]]
[1]  6 10 12

[[149]]
[1] 4 7 2

[[150]]
[1] 11 10  3

[[151]]
[1] 12  7  9

[[152]]
[1] 10  6  3

[[153]]
[1]  8 11  1

[[154]]
[1]  9  3 10

[[155]]
[1]  1  3 10

[[156]]
[1]  1 11  2

[[157]]
[1]  4 12  9

[[158]]
[1] 7 9 4

[[159]]
[1] 3 6 5

[[160]]
[1] 2 6 5

[[161]]
[1] 8 7 1

[[162]]
[1] 3 4 6

[[163]]
[1] 11  7  9

[[164]]
[1]  7 12  4

[[165]]
[1] 10  4 12

[[166]]
[1] 9 1 5

[[167]]
[1]  6  4 11

In [57]:
bobot = function(pilihan,course){
  pilihan = unlist(pilihan)
  ikang = which(pilihan == course)
  ikang = ifelse(length(ikang) == 0, -1000, 4-ikang)
  return(ikang)
}
pilihan[1]
bobot(pilihan[1],1)

[[1]]
[1]  9 12  7

[1] -1000

In [58]:
model <- MIPModel() %>%
  
  # 1 iff student i is assigned to course m
  add_variable(x[i, j], i = 1:n, j = 1:m, type = "binary") %>%
  
  # maximize the preferences
  set_objective(sum_expr(bobot(pilihan[i], j) * x[i, j], i = 1:n, j = 1:m),
                "max") %>%
  
  # we cannot exceed the capacity of a course
  add_constraint(sum_expr(x[i, j], i = 1:n) <= capacity[j], j = 1:m) %>% 
  
  # each student needs to be assigned to one course
  add_constraint(sum_expr(x[i, j], j = 1:m) == 1, i = 1:n)
model

Mixed integer linear optimization problem
Variables:
  Continuous: 0 
  Integer: 0 
  Binary: 2004 
Model sense: maximize 
Constraints: 179 

In [59]:
result <- solve_model(model, with_ROI(solver = "glpk", verbose = TRUE))
result

<SOLVER MSG>  ----
GLPK Simplex Optimizer, v4.65
179 rows, 2004 columns, 4008 non-zeros
      0: obj =  -0.000000000e+00 inf =   1.670e+02 (167)
    177: obj =  -1.249210000e+05 inf =   0.000e+00 (0)
Perturbing LP to avoid stalling [610]...
Removing LP perturbation [724]...
*   724: obj =   4.930000000e+02 inf =   0.000e+00 (0) 2
OPTIMAL LP SOLUTION FOUND
GLPK Integer Optimizer, v4.65
179 rows, 2004 columns, 4008 non-zeros
2004 integer variables, all of which are binary
Integer optimization begins...
Long-step dual simplex will be used
+   724: mip =     not found yet <=              +inf        (1; 0)
+   724: >>>>>   4.930000000e+02 <=   4.930000000e+02   0.0% (1; 0)
+   724: mip =   4.930000000e+02 <=     tree is empty   0.0% (0; 1)
INTEGER OPTIMAL SOLUTION FOUND
<!SOLVER MSG> ----


Status: optimal
Objective value: 493

In [60]:
final = 
  result %>% 
  get_solution(x[i,j]) %>%
  filter(value > 0) %>%
  rename(id_peserta = i,
         batch_ke = j) %>%
  select(-variable,-value)

final
table(final$batch_ke)

id_peserta,batch_ke
<int>,<int>
2,1
14,1
19,1
44,1
97,1
107,1
122,1
123,1
126,1



 1  2  3  4  5  6  7  8  9 10 11 12 
16  9 10 14 15 13 16 15 16 16 11 16 

In [61]:
#install.packages("ROI")
#install.packages("ompr")
#install.packages("ompr.roi")
#system("sudo apt-get install libglpk-dev")
#install.packages("ROI.plugin.glpk")
save(pilihan,final,bobot,file = "bahan blog.rda")